<center>
    <h1 style="color:deeppink">Just between us SQL Friends...</h1>
    <b style="color:deeppink">RuPaul's Drag Race: Visualized<b>
    <img src="media/header.gif" height="100px">
</center>

<h2 style="color:deeppink">Project Requirements</h2>

<ul>
    <li><strike>You must include a SQL database (MySQL or SQLite) where your data will be stored</strike></li>
    <li><strike>You need to include the sql or python script that creates your database</strike></li>
    <li><strike>You must include a Python script used to fetch data from a data source and load it into your SQL database</strike></li>
    <li><strike>Your data source may be an external API, database, a CSV file, JSON, or any other data source that you can read/parse via Python code</strike></li>
    <li><strike>You must include a Python script to retrieve the data from your SQL database into a Python object</strike></li>
    <li>Visualize the results of your analysis using Matplotlib, Seaborn, Bokeh or another Python Data </li>Visualization library. Your results can not be a plain text representation and you are encouraged to explore a visualization approach that clearly supports a conclusion/result of the analysis of your data.</li>
    <li>Your data retrieval for visualization uses at least one SQL query, meaning you can't parse records from your data set using only Python or an ORM. </li>
</ul>

<h2 style="color:deeppink">Data Schema</h2>
<ul>
    <li>
        <h3 style="color:deeppink">queens_df</h3>
        <ul>
            <li>id - int</li>
            <li>name - str</li>
            <li>quote - str</li>
            <li>image_url - str</li>
            <li>miss_congeniality - bool</li>
            <li>winner - bool</li>
        </ul>
    </li>
    <li>
        <h3 style="color:deeppink">main_seasons_df</h3>  
        <ul>
            <li>season_id - int</li>
            <li>queen_id - int</li>
            <li>place - int</li>
        </ul>
    </li> 
    <li>
        <h3 style="color:deeppink">all_stars_df</h3>
        <ul>
            <li>season_id - int</li>
            <li>queen_id - int</li>
            <li>place - int</li>
        </ul>
    </li>
</ul>

<h2 style="color:deeppink">To-Do:</h2>

<ul>
    <li></li>
    <li></li>    
    <li></li>
</ul>

<h2 style="color:deeppink">Import Libraries & Connect SQLite Database</h2>

In [98]:
import pandas as pd
import sqlite3
from IPython.display import Image
from pandas.io.json import json_normalize

drag_race_data_base = sqlite3.connect("drag_race_data.db")

<h2 style="color:deeppink">Define Constants, Global Variables, and API endpoints</h2>

In [99]:
#API URL
DRAG_RACE_API_URL = "http://www.nokeynoshade.party/api/"

#API Endpoints - may not need all of these in the finished product
SEASONS = "seasons"
SEASON_BY_NUMBER = "seasons/{}"
QUEENS = "queens/all"
QUEENS_BY_SEASON = "seasons/{}/queens"
EPISODES = "episodes"
EPISODES_BY_SEASON = "seasons/{}/episodes"
CHALLENGES_BY_SEASON = "seasons/{}/challenges"
WINNERS = "queens/winners"
CHALLENGES = "challenges"

<h2 style="color:deeppink">Define Functions</h2>

In [100]:
def get_data(endpoint, SEASON = None):
    data = pd.read_json(DRAG_RACE_API_URL + endpoint.format(SEASON))
    return data

def get_seasons():
    season_queens_df = pd.DataFrame()
    temp_df = pd.DataFrame()
    for i in range(len(season_id_df)):
        temp_df = get_data(SEASON_BY_NUMBER, i + 1)
        temp_df = json_normalize(temp_df['queens'])
        temp_df['season_id'] = i + 1
        season_queens_df = pd.concat([season_queens_df, temp_df], sort=False)
        i += 1
    return season_queens_df


<h2 style="color:deeppink">Define SQL Queries</h2>

In [168]:
queens_query = """
    SELECT season_id, queens.id, name, image_url FROM seasons
    INNER JOIN queens ON seasons.id = queens.id
    WHERE winner IS True
"""

<h2 style="color:deeppink">Create raw dataframes from API endpoints</h2>

In [169]:
raw_seasons_df = get_data(SEASONS)
raw_queens_df = get_data(QUEENS)

<h2 style="color:deeppink">Define values used for cleanup</h2>

In [170]:
queens_drop_cols = ['queens', 'name']

<h2 style="color:deeppink">Clean and organize the data</h2>

In [171]:
# create the season_id_df to look up season numbers by id
season_id_series = raw_seasons_df['id']
season_number_series = raw_seasons_df['seasonNumber']
seasons_dict = {"season_id":season_id_series, "season_number": season_number_series}
season_id_df = pd.DataFrame(seasons_dict)

<h2 style="color:deeppink">Create SQL Tables from raw dataframes</h2>

In [172]:
raw_queens_df.to_sql('queens', drag_race_data_base, if_exists='replace')

seasons_df = get_seasons().set_index('id').drop(queens_drop_cols, axis=1)
seasons_df.to_sql('seasons', drag_race_data_base, if_exists = 'replace')

<h2 style="color:deeppink">Sandbox</h2>
<b style="color:deeppink">Use the space below to experiment. Nothing below will be included in the finished product.</b>

In [173]:
pd.read_sql(queens_query, drag_race_data_base)

,season_id,id,name,image_url
0,1,12,Bebe Zahara Benet,http://www.nokeynoshade.party/images/bebe-zaha...
1,12,12,Bebe Zahara Benet,http://www.nokeynoshade.party/images/bebe-zaha...
2,2,24,Tyra Sanchez,http://www.nokeynoshade.party/images/tyra-sanc...
3,3,36,Raja,http://www.nokeynoshade.party/images/raja.jpg
4,4,48,Chad Michaels,http://www.nokeynoshade.party/images/chad-mich...
5,5,48,Chad Michaels,http://www.nokeynoshade.party/images/chad-mich...
6,4,49,Sharon Needles,http://www.nokeynoshade.party/images/sharon-ne...
7,6,62,Alaska Thunderfuck 5000,http://www.nokeynoshade.party/images/alaska-th...
8,10,62,Alaska Thunderfuck 5000,http://www.nokeynoshade.party/images/alaska-th...
9,6,63,Jinkx Monsoon,http://www.nokeynoshade.party/images/jinkx-mon...


In [109]:
# winners_query = """
#     SELECT id, name, quote, image_url
#     FROM queens
#     WHERE winner IS True
# """

# winners_df = pd.read_sql(winners_query, drag_race_data_base).set_index('id')
# winners_df